# Using DC2 and CosmoDC2 to Extract Shear and Convergence #

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

note: commented portions of cells below to illustrate attempted methods that were suboptimal (esp. from dc2-linked tutorials), included for future observation

In [2]:
import torch
import numpy as np
from os import environ
from pathlib import Path
from einops import rearrange

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

from hydra import initialize, compose
from hydra.utils import instantiate
import healpy as hp

In [3]:
with initialize(config_path=".", version_base=None):
    cfg = compose("config")
train_dc2_cfg = cfg.copy()

CosmoDC2 is REALLY big so we don't want to load all of its data into memory when accessing attributes. We set filters below that match dc2's piece of the sky since it's a subset of Cosmo's coverage. In the below cell, we load our DC2 catalog with the fields we are interested in and find the relevant sky quadrant, allowing us to build the filters.

In [4]:
import GCRCatalogs
from GCRCatalogs import GCRQuery
GCRCatalogs.set_root_dir("/data/dc2/")
truth_cat = GCRCatalogs.load_catalog('desc_dc2_run2.2i_dr6_truth')
truth_data = truth_cat.get_quantities(["id", "cosmodc2_id", "match_objectId", "ra", "dec", "truth_type"])

max_ra = np.nanmax(truth_data['ra'])
min_ra = np.nanmin(truth_data['ra'])
max_dec = np.nanmax(truth_data['dec'])
min_dec = np.nanmin(truth_data['dec'])
pos_filters = [f'ra >= {min_ra}',f'ra <= {max_ra}', f'dec >= {min_dec}', f'dec <= {max_dec}']

vertices = hp.ang2vec(np.array([min_ra, max_ra, max_ra, min_ra]),
                      np.array([min_dec, min_dec, max_dec, max_dec]), lonlat=True)
ipix = hp.query_polygon(32, vertices, inclusive=True)
healpix_filter = GCRQuery((lambda h: np.isin(h, ipix, True), "healpix_pixel"))

print(truth_data.keys())

dict_keys(['truth_type', 'match_objectId', 'ra', 'cosmodc2_id', 'dec', 'id'])


Next, load in CosmoDC2 and select the quantities we want bounded by the aforementioned filters

In [5]:
config_overwrite = dict(
    catalog_root_dir='/data/dc2/cosmoDC2'
)

cosmo_cat = GCRCatalogs.load_catalog('desc_cosmodc2', config_overwrite)

In [6]:
cosmo_data = cosmo_cat.get_quantities(quantities = ["galaxy_id", "ra", "dec", "shear_1", "shear_2", "convergence"], filters=pos_filters, native_filters=healpix_filter)
print(cosmo_data.keys())

dict_keys(['shear_2', 'ra', 'galaxy_id', 'dec', 'convergence', 'shear_1'])


#### Matching Attempt 1: Using example code from ####
https://github.com/LSSTDESC/DC2-analysis/blob/253625a230d545f4ceb529aae58416ef7a768648/tutorials/matching_fof.ipynb

Note: Do not actually run the commented cells below, takes forever and does not converge.

In [7]:
# cosmo = pd.DataFrame(cosmo_data)
# tru = pd.DataFrame(truth_data)
# import FoFCatalogMatching
# results = FoFCatalogMatching.match(
#     catalog_dict={'truth': tru, 'object': cosmo}, 
#     linking_lengths=1.0,
# )

In [8]:
# truth_mask = results['catalog_key'] == 'truth'
# object_mask = ~truth_mask

# # then np.bincount will give up the number of id occurrences (like historgram but with integer input)
# n_groups = results['group_id'].max() + 1
# n_truth = np.bincount(results['group_id'][truth_mask], minlength=n_groups)
# print(n_truth[n_truth>10])
# n_object = np.bincount(results['group_id'][object_mask], minlength=n_groups)

# # now n_truth and n_object are the number of truth/object objects in each group
# # we want to make a 2d histrogram of (n_truth, n_object). 
# n_max = max(n_truth.max(), n_object.max()) + 1
# hist_2d = np.bincount(n_object * n_max + n_truth, minlength=n_max*n_max).reshape(n_max, n_max)

# plt.imshow(np.log10(hist_2d+1), extent=(-0.5, n_max-0.5, -0.5, n_max-0.5), origin='lower');
# plt.xlabel('Number of truth objects');
# plt.ylabel('Number of object objects');
# plt.colorbar(label=r'$\log(N_{\rm groups} \, + \, 1)$');

In [9]:
# # Let's further inspect the objects in the groups that have 1-to-1 truth/object match.

# # first, let's find our the IDs of the groups that have 1-to-1 truth/object match:
# one_to_one_group_mask = np.in1d(results['group_id'], np.flatnonzero((n_truth == 1) & (n_object == 1)))

# # and then we can find the row indices in the *original* truth/object catalogs for those 1-to-1 groups
# truth_idx = results['row_index'][one_to_one_group_mask & truth_mask]
# object_idx = results['row_index'][one_to_one_group_mask & object_mask]

In [10]:
# with open("friends.pkl", "wb") as f:
#     pickle.dump(results, f)
# print("saved friends")

#### Now, since that approach didn't work, let's try something that was attempted by @Xinyue: ####
https://github.com/prob-ml/bliss/blob/master/case_studies/dc2/DC2_galaxy_psf_params.ipynb

merging on galaxy_id and cosmodc2_id


In [11]:
import pandas as pd
df_data = pd.DataFrame(truth_data)
df_galaxy = pd.DataFrame(cosmo_data)

merge_data = df_data.merge(
    df_galaxy, 
    left_on = "cosmodc2_id", 
    right_on = "galaxy_id", 
    how = "inner" 
)
merge_data_fill = merge_data.dropna()

In [12]:
merge_data.to_csv('/data/scratch/shreyasc/combined_idmatch_truth.csv')

In [13]:
merge_data.shape

(43408507, 12)

#### Now, since we actually just care about shear and convergence at locations defined by {ra, dec}, let's try explicitly merging on those ####

In [14]:
ra_dec_explicit_merge = pd.merge(df_data, df_galaxy, on=["ra", "dec"], how='inner')

In [ ]:
ra_dec_explicit_merge.to_csv('/data/scratch/shreyasc/combined_ra_dec_merge_truth.csv')

In [ ]:
ra_dec_explicit_merge.shape

(43408508, 10)

In [ ]:
merge_data.head()

,id,ra_x,truth_type,dec_x,match_objectId,cosmodc2_id,ra_y,convergence,shear_1,shear_2,dec_y,galaxy_id
0,8757401501,56.851958,1,-38.677521,15982501021353368,8757401501,56.851958,-0.022032,-0.022370,0.041324,-38.677521,8.757402e+09
1,8750694182,56.799616,1,-38.677104,15982501021353393,8750694182,56.799616,0.004835,-0.000660,-0.000362,-38.677104,8.750694e+09
2,8752812073,56.883238,1,-38.676882,15982501021353397,8752812073,56.883238,-0.006825,-0.009312,0.014221,-38.676882,8.752812e+09
3,8750045581,56.858170,1,-38.676674,15982501021353414,8750045581,56.858170,-0.000179,-0.001958,0.000761,-38.676674,8.750046e+09
4,8751841300,56.889715,1,-38.676423,15982501021353419,8751841300,56.889715,-0.004813,-0.008540,0.007185,-38.676423,8.751841e+09


In [ ]:
ra_dec_explicit_merge.head()

,id,ra,truth_type,dec,match_objectId,cosmodc2_id,convergence,shear_1,shear_2,galaxy_id
0,8757401501,56.851958,1,-38.677521,15982501021353368,8757401501,-0.022032,-0.022370,0.041324,8757401501
1,8750694182,56.799616,1,-38.677104,15982501021353393,8750694182,0.004835,-0.000660,-0.000362,8750694182
2,8752812073,56.883238,1,-38.676882,15982501021353397,8752812073,-0.006825,-0.009312,0.014221,8752812073
3,8750045581,56.858170,1,-38.676674,15982501021353414,8750045581,-0.000179,-0.001958,0.000761,8750045581
4,8751841300,56.889715,1,-38.676423,15982501021353419,8751841300,-0.004813,-0.008540,0.007185,8751841300


#### Sanity Check of the ra / dec from the initial merge ####

In [ ]:
(merge_data["ra_x"] - merge_data["ra_y"]).mean()

0.0

In [ ]:
(merge_data["dec_x"] - merge_data["dec_y"]).mean()

0.0

Note via @declan: redshift in cosmoDC2 and DC2 truth table were significantly different for common id's. Could be a possible bug in the id-matching.